In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

/Users/h.ren/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import load_model
model = load_model('saved_models/Emotion_Voice_Detection_Model.h5')

In [ ]:
# Audio Recorder

import pyaudio
import wave

CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2 
RATE = 44100 #sample rate
RECORD_SECONDS = 4
WAVE_OUTPUT_FILENAME = "live_test.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording


In [22]:
import os
import pandas as pd
import librosa
import glob

df = pd.DataFrame(columns=['feature'])
bookmark=0

X, sample_rate = librosa.load('live_test.wav', res_type='kaiser_fast', duration=2.5, sr=22050*2, offset=0.5)
sample_rate = np.array(sample_rate)
librosa_feature = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13)
mfccs = np.mean(librosa_feature, axis=0)
feature = mfccs
#[float(i) for i in feature]
#feature1=feature[:135]
df.loc[bookmark] = [feature]
bookmark=bookmark+1        

In [25]:
test_set = np.expand_dims(pd.DataFrame(df['feature'].values.tolist()), axis=2)

In [34]:
preds = model.predict(test_set, batch_size=32, verbose=1)

1/1 [==============================] - 0s 4ms/step


In [43]:
res = preds.argmax(axis=1).astype(int)

In [50]:
mapping = {
    0:"male_calm",
    1:"male_sad",
    2:"male_fearful",
    3:"female_fearful",
    4:"male_fearful",
    5:"male_happy",
    6:"female_calm",
    7:"female_angry",
    8:"male_angry",
    9:"male_happy"
}
mapping[res[0]]

'male_happy'